# Visualisation des arbres à Grenoble

In [1]:
import folium
import pandas as pd
import numpy as np
import json
from  geopy import distance
import math

In [2]:
file_path="../data/trees.csv"
df_tree = pd.read_csv(file_path, sep=",", header = 0, index_col=False,names=None)

### Extraction données GPS en Json

In [3]:
df_tree_json = pd.json_normalize(df_tree.GeoJSON.apply(json.loads))
df_tree  = pd.concat([df_tree,df_tree_json],axis=1)
df_tree[['longitude','latitude']] = pd.DataFrame(df_tree['coordinates'].to_list(),columns=['longitude','latitude'])

### Creation de la carte

In [5]:
location_gre=[45.166672,5.71667]
grenoble = folium.Map(location = location_gre,zoom_start=13)

In [58]:
%%time
for i in range(len(df_tree)):
    loc = [df_tree.iloc[i]['latitude'],df_tree.iloc[i]['longitude']]
    folium.Circle(loc,radius=1,fill_opacity=0.8).add_to(grenoble)

CPU times: user 7.59 s, sys: 88.6 ms, total: 7.68 s
Wall time: 7.55 s


In [7]:
grenoble.save("grenoble.html")

### Calcul de distances

#### Module geopy & Haversine

In [8]:
def haversine(lat_a,long_a,lat_b,long_b):
    lat_a, lat_b, long_a, long_b = map(np.deg2rad, [lat_a, lat_b, long_a, long_b])
    
    a = (np.sin((long_b-long_a)/2))**2
    a *= np.cos(lat_a) * np.cos(lat_b)
    a += (np.sin((lat_b-lat_a)/2))**2
    c = 2 * np.arcsin(np.sqrt(a))
    d = 6371 * c
    return d

def dist_geo(lat_a,long_a,lat_b,long_b):
    t_a =(lat_a,long_a)
    t_b =(lat_b,long_b)
    dist = distance.distance(t_a,t_b)
    return dist
    

#### Test calcul des distances

In [9]:
dist = haversine(df_tree['latitude'][0],df_tree['longitude'][0],df_tree['latitude'][1],df_tree['longitude'][1])
print(dist)
dist = dist_geo(df_tree['latitude'][0],df_tree['longitude'][0],df_tree['latitude'][1],df_tree['longitude'][1])
print(dist)

0.05062970404697751
0.05071257785524804 km


### Les arbres les plus proches de chez moi

Explication du code

On va tester la fonction apply ainsi que l'utilisation d'une fonction vectorielle pour verifier la vitesse de calcul avec ses deux opérations.
Sources :
. Sofia Heisler No More Sad Pandas Optimizing Pandas Code for Speed and Efficiency PyCon 2017
. 1000x faster data manipulation: vectorizing with Pandas and Numpy PyGotham 2019

In [10]:
def dist_tohome(lat_a,long_a):
    home = (45.19101638257626, 5.724418886125648)
    d = haversine(lat_a,long_a,45.19101638257626, 5.724418886125648)
    return d

In [11]:
# test distance maison
maison=(45.19101638257626, 5.724418886125648)
print(dist_tohome(df_tree['latitude'][0],df_tree['longitude'][0]))
print(dist_tohome(df_tree['latitude'][1],df_tree['longitude'][1]))
print(dist_tohome(df_tree['latitude'][2],df_tree['longitude'][2]))

4.958400121800038
4.960151591750655
4.969341428822664


In [12]:
# utilisation vectorisation pandas

In [13]:
%%timeit
df_tree['distance2home'] = dist_tohome(df_tree['latitude'],df_tree['longitude'])

2.38 ms ± 54.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
# utilisation fonction iterrows

In [15]:
%%timeit
distances = []
for index,row in df_tree.iterrows():
    distances.append(dist_tohome(row['latitude'],row['longitude']))
df_tree['distance2home'] = distances

1.69 s ± 29 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
# utilisation fonction apply

In [17]:
%%timeit
df_tree['distance2home'] = df_tree.apply(lambda row : dist_tohome(row['latitude'],row['longitude']), axis =1)

625 ms ± 4.01 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
# afficher les 10 arbres les plus proches de chez moi 

On trie les valeurs par la distance et on récupère par une boucle les 10 premières pour l'ajouter à une carte

In [19]:
df_tree.sort_values(['distance2home'],axis=0,inplace=True)

In [20]:
location_home=[45.19108972346367, 5.724318356006014]
home_map = folium.Map(location = location_home,zoom_start=14)
folium.Marker(location_home,
              popup="Maison",
              icon=folium.Icon(icon='home',prefix='fa',color='blue'),).add_to(home_map)

for i in range(0,10):
    loc = [df_tree.iloc[i]['latitude'],df_tree.iloc[i]['longitude']]
    folium.Circle(loc,radius=2,color= 'green').add_to(home_map)

# home_map

### Distance moyenne entre les arbres

Explication du code

Pour chaque arbre, calcul de la distance avec tous les autres et ensuite récupération de la plus petite distance (qui n'est pas zero) 
Stock des distances mini dans liste et calcul des distances moyennes à partir de ces données. 
Run time 2min 35sec

In [21]:
%%time
dist_moy = []
lat_arb = 0
long_arb = 0
for i in range(len(df_tree)):
    df_tree['dist_tree'] = haversine(df_tree['latitude'],df_tree['longitude'],df_tree.iloc[i]['latitude'],df_tree.iloc[i]['longitude'])
    dist_moy.append(df_tree['dist_tree'].sort_values(axis=0,ascending=True).iloc[1])

CPU times: user 2min 33s, sys: 27.7 ms, total: 2min 33s
Wall time: 2min 33s


In [22]:
print(f"La distance moyenne entre les arbres voisins est de {sum(dist_moy)*1000/len(dist_moy):.1f} metres")

La distance moyenne entre les arbres voisins est de 6.2 metres


In [23]:
df_tree['distarbreproche'] = dist_moy

### Arbre le plus isolé

Arbre avec la distance à l'arbre le plus proche la plus grande

In [24]:
lat_iso = df_tree.loc[df_tree['distarbreproche'].idxmax(),'latitude']
long_iso = df_tree.loc[df_tree['distarbreproche'].idxmax(),'longitude']
loc_iso = (lat_iso,long_iso)

In [25]:
folium.Marker(loc_iso,radius=2,color= 'green',
              popup="Arbre le plus isolé",
              icon=folium.Icon(icon='tree',prefix='fa',color='green'),).add_to(home_map)
home_map

### Arbre le plus entouré

Difficile à réaliser en brute force ...

### Utilisation de KDTree

#### Creation du KDTree

In [26]:
import scipy 
import scipy.spatial

In [27]:
df_kd = df_tree[['latitude','longitude']]

In [28]:
kd = scipy.spatial.KDTree(df_kd[['latitude','longitude']].values)
kd

#### Arbres les plus proche de chez moi

In [29]:
maison=[45.19101638257626, 5.724418886125648]
dd, ii = kd.query(maison,k=10)
print(dd," ",ii)

[0.00073922 0.00082921 0.00082962 0.00085087 0.00087451 0.00087671
 0.00089028 0.00089596 0.00094488 0.0009786 ]   [ 0  4  1  6  3  9  7 11  2 14]


On retrouve des arbres similaires de ceux par la méthode brute mais il y a quelques différences (indices 11 et 14) 
Par contre le temps d'execution est beaucoup plus rapide par rapport à la méthode ou on calcule la distance pour tout les points. 22µs vs 2.9 ms (vectorielle pandas)

#### Distance entre les arbres

Pour chaque arbre on recherche le voisin le plus près (k=2 car sinon c'est lui même) , on stocke dans une liste et ensuite on fait la moyenne de cette distance

In [30]:
%%time
liste_distance=[]
liste_indice=[]
for i in range(len(df_tree)):
    distance=0
    lat_a = df_tree.iloc[i]['latitude']
    long_a = df_tree.iloc[i]['longitude']
    arb_loc=[lat_a,long_a]
    d, ind = kd.query(arb_loc,k=2)
    indice = ind[1]
    lat_n = df_tree.iloc[indice]['latitude']
    long_n = df_tree.iloc[indice]['longitude']
    distance = haversine(lat_n,long_n,lat_a,long_a)
    liste_distance.append(distance)
    liste_indice.append(i)
moyenne_dist = sum(liste_distance)/len(liste_distance)
print(f"La distance moyenne entre deux arbres est de {moyenne_dist*1000:.1f} metres")

La distance moyenne entre deux arbres est de 6.3 metres
CPU times: user 14.5 s, sys: 16.1 ms, total: 14.5 s
Wall time: 14.4 s


#### Arbre le plus entouré/ le plus solitaire

##### Arbre solitaire à partir de la distance mini au prochain arbre

In [31]:
distance_max = max(liste_distance)
index_max = liste_distance.index(distance_max)

lat_iso = df_tree.iloc[index_max]['latitude']
long_iso = df_tree.loc[index_max]['longitude']
loc_iso = (lat_iso,long_iso)

folium.Marker(loc_iso,radius=2,color= 'green',
              popup="Isolé_KD_distance",
              icon=folium.Icon(icon='tree',prefix='fa',color='green'),).add_to(home_map)

On recherche l'arbre avec le plus de voisins dans un rayon de 20 m (~2x la distance moyenne entre deux arbres) et inverse pour le plus solitaire.
Itération du la data_frame et KDTree.query_ball_tree
ou utilisation de query pairs qui cherche les paires d'arbres dans le kdtree dans une distance donnée et on récupère ensuite l'arbre le plus nommé.

In [32]:
rayon = 0.0001

##### Methode query_pairs

On recupère un array et on stocke le compte des uniques dans un dictionnaire

In [33]:
%%time

pair = kd.query_pairs(rayon,output_type='ndarray')
unique, counts = np.unique(pair, return_counts=True)
dico_voisin = dict(zip(unique, counts))
entoure = max(dico_voisin,key=dico_voisin.get)
print(entoure, " " , dico_voisin[entoure])

11264   70
CPU times: user 18.2 ms, sys: 0 ns, total: 18.2 ms
Wall time: 17.7 ms


##### Methode query_ball_point

In [34]:
# test fonction

lat_a = df_tree.iloc[11283]['latitude']
long_a = df_tree.iloc[11283]['longitude']
nb_voisin = kd.query_ball_point([lat_a,long_a],r=rayon,return_length=True)
print(nb_voisin)

41


In [35]:
%%time

array_kd = kd.query_ball_point(df_tree[['latitude','longitude']],r=rayon,return_length=True)
array_kd

CPU times: user 45 ms, sys: 0 ns, total: 45 ms
Wall time: 44.4 ms


array([1, 1, 2, ..., 1, 3, 2])

Même methode que au dessus mais avec une for loop for pas belle et moins rapide 

In [36]:
%%time


list_voisin = []
for i in range(len(df_tree)):
    lat_a = df_tree.iloc[i]['latitude']
    long_a = df_tree.iloc[i]['longitude']
    nb_voisin = kd.query_ball_point([lat_a,long_a],r=rayon,return_length=True)
    tu = (nb_voisin,i)
    list_voisin.append(tu)

CPU times: user 7.1 s, sys: 16 ms, total: 7.12 s
Wall time: 7.14 s


In [37]:
arbre_entoure = max(list_voisin)[1]
arbre_solo = min(list_voisin)[1]

Ajout des deux arbres sur la carte

In [38]:
lat_iso = df_tree.iloc[arbre_solo]['latitude']
long_iso = df_tree.loc[arbre_solo]['longitude']
loc_iso = (lat_iso,long_iso)

lat_entou = df_tree.iloc[arbre_entoure]['latitude']
long_entou = df_tree.loc[arbre_entoure]['longitude']
loc_entou = (lat_entou,long_entou)

In [39]:
folium.Marker(loc_iso,radius=2,color= 'green',
              popup="Arbre le plus isolé_KD",
              icon=folium.Icon(icon='tree',prefix='fa',color='green'),).add_to(home_map)
folium.Marker(loc_entou,radius=2,color= 'green',
              popup="Arbre le plus entouré_KD",
              icon=folium.Icon(icon='tree',prefix='fa',color='green'),).add_to(home_map)

### La terre n'est pas plate ? BallTree

Soit on converti les latitudes longitudes pour quelles fonctionnent sur un plan
Ou utilisation de ball tree (module sklearn) qui prend en compte directement les coordonnées sphériques.

#### BallTree

In [40]:
from sklearn.neighbors import BallTree

In [41]:
## array numpy et convertir en radians
arbres_gps = df_kd[['latitude','longitude']].values
arbres_radian = np.radians(arbres_gps)
ball = BallTree(arbres_radian,metric='haversine')

In [42]:
df_kd[['latitude','longitude']].values

array([[45.19152834,  5.72388564],
       [45.19151056,  5.7237525 ],
       [45.1914885 ,  5.72360041],
       ...,
       [45.15307606,  5.76030815],
       [45.15310648,  5.76041164],
       [45.15312431,  5.76050852]])

In [43]:
maison = np.array([[45.19101638257626,5.724418886125648]])
maison = np.radians(maison)

In [44]:
distance,indice = ball.query(maison,k=2)
distance = distance * 6371 *1000

On trouve des valeurs logiques avec google maps (70 metre par exemple).

In [45]:
distance[0][1]

75.80518093638725

#### Distance entre les arbres BallTree

In [46]:
%%time

liste_dist=[]
for i in range(len(df_tree)):
    d=0
    distance=0
    arbre = np.array([[df_tree.iloc[i]['latitude'],df_tree.iloc[i]['longitude']]])
    arbre_rad = np.radians(arbre)
    d, i = ball.query(arbre_rad,k=2)
    liste_dist.append(d[0][1])

CPU times: user 9.49 s, sys: 0 ns, total: 9.49 s
Wall time: 9.49 s


In [47]:
for i,dist in enumerate(liste_dist):
    liste_dist[i] = dist * 6371
moyenne_dist = sum(liste_dist)/len(liste_dist)
print(f"La distance moyenne entre deux arbres est de {moyenne_dist*1000:.1f} metres")

La distance moyenne entre deux arbres est de 6.2 metres


#### Arbre le plus entouré/ le plus solitaire

On recherche l'arbre avec le plus de voisins dans un rayon de 10 m  et inverse pour le plus solitaire.
ball.query_radius ne fonctionne pas en mode itération arbre par arbre, il faut lui passer directement les coordonnées pour tout les arbres et on se débrouille après

In [48]:
rayon = 20/6371000

In [49]:
%%time

df_kd_rad = df_kd[['latitude','longitude']]
df_kd_rad = np.radians(df_kd_rad)

array = ball.query_radius(np.column_stack((df_kd_rad.latitude.values,df_kd_rad.longitude.values)),rayon)

CPU times: user 276 ms, sys: 0 ns, total: 276 ms
Wall time: 276 ms


In [50]:
array

array([array([1, 0]), array([1, 5, 2, 0]), array([ 1,  5,  2, 10, 12]),
       ..., array([31238, 31236, 31242, 31237, 31239, 31241, 31240]),
       array([31238, 31236, 31242, 31237, 31239, 31241, 31240]),
       array([31238, 31236, 31242, 31239, 31241, 31240])], dtype=object)

In [51]:
listoflen = np.array([len(array) for array in array])
entou = listoflen.argmax()
listoflen.max()

116

Afficher sur la carte l'arbre avec ball_tree_voisin

In [52]:
entou = listoflen.argmax()
lat_entou = df_kd.iloc[entou]['latitude']
long_entou = df_kd.iloc[entou]['longitude']
loc_entou = (lat_entou,long_entou)

folium.Marker(loc_entou,radius=2,color= 'green',
              popup="entouré_ballTree",
              icon=folium.Icon(icon='tree',prefix='fa',color='green'),).add_to(home_map)

# cette méthode pour le plus solitaire ne fonctionne pas vraiment car il ne discrimine pas entre les arbres solitaires. 
# La méthode par la distance est plus juste

solo = listoflen.argmin()

lat_iso = df_kd.iloc[solo]['latitude']
long_iso = df_kd.iloc[solo]['longitude']
loc_iso = (lat_iso,long_iso)

folium.Marker(loc_iso,radius=2,color= 'green',
              popup="solo_ballTree",
              icon=folium.Icon(icon='tree',prefix='fa',color='green'),).add_to(home_map)

### Conclusion

In [53]:
home_map

Concernant la distance, on retrouve des valeurs équivalentes entre les 3 méthodes : bruteforce haversine, kdtree haversine et ball tree haversine.
Par contre pour les arbres isolés ou entourés, les valeurs ne convergent pas...

Pour le plus isolé, la méthode brute sur la distance semble la meilleure selon les données OpenStreetMap
Pour le plus entouré, les valeurs diffèrent en fonction du rayon choisi.
Avec KDTree, la conversion des distances n'est pas directe 
Avec KDBall de sklearns, la conversion distance est réalisable plus facilement(division par rayon terrestre).